In [1]:
# %load std_libraries.py
#to keep my scripts consistent, I am adding this to the top of all scripts with %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = other")
location_dict = {'a': "C:\\Users\\BMH_work\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",
                 'c': "C:\\Users\\Ben\\Documents\\GitHub\\expression_broad_data", 'd':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.colormap as cm
#from matplotlib_venn import venn2
#for my windows10 laptop I had to install this package using pip rather than anaconda.  
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
#import scipy.stats as stats
#import statsmodels.graphics.gofplots as stats_graph
#import scipy.cluster.hierarchy as sch

# from Bio import SeqIO
# from Bio import SeqFeature as sf
# from Bio.Alphabet import generic_dna
# from Bio.Seq import Seq

# import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#from itertools import product
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
    
# for phylogenetic trees: 
# from ete3 import Tree
#ete3 is not officially supported on windows, and so must be loaded via pip: 
# pip install -U https://github.com/etetoolkit/ete/archive/qt5.zip
# ref: https://groups.google.com/forum/#!topic/etetoolkit/6NblSBPij4o

#for scraping internet data (e.g. ncbi)
#import requests
#from lxml import etree    #parses xml output

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = othera
base directory is C:\Users\BMH_work\github\expression_broad_data
Added C:\Users\BMH_work\github\expression_broad_data to path: 
['', 'C:\\Users\\BMH_work\\github\\expression_broad_data\\scripts', 'C:\\Users\\BMH_work\\Anaconda3\\python37.zip', 'C:\\Users\\BMH_work\\Anaconda3\\DLLs', 'C:\\Users\\BMH_work\\Anaconda3\\lib', 'C:\\Users\\BMH_work\\Anaconda3', 'C:\\Users\\BMH_work\\Anaconda3\\lib\\site-packages', 'C:\\Users\\BMH_work\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\BMH_work\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\BMH_work\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\BMH_work\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\BMH_work\\.ipython', 'C:\\Users\\BMH_work\\github\\expression_broad_data']
I am about to import a library
I am importing io_library
what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = other

In [2]:
#Compare both species AS +/- drug

#eventually I might want a big database here but for now just going to load what I need. 


#Load KL to SC ortholog mapping

kl_orthologs = pd.read_pickle(data_processing_dir + "ortholog_files_YGOB/kl_orthologs.pkl")

#Load data for each species and combine desired data into one dataframe per species. 

lfc_data = {}
# rlog_data = {}
# raw_data = {}
comparison_data = {}

species = ['KL', 'SC']
# conditions = ["AS_noNMPP1_YPD_t40", "AS_yesNMPP1_YPD_t40"]
for spec in species: 
    #Load DEseq data for AS +/- drug
    lfc_data_spec = pd.read_csv(os.path.normpath(data_processing_dir + '\\kl_sc_PKA_as_m24_r1g1_20181017\\20181017_deseq_' + spec + '_AS_nmpp1.csv'), index_col=0)
    if spec == 'KL':
        kl_genenames = io_library.kl_genename_convert_list(lfc_data_spec.index)
        lfc_data_spec['kl_genename'] = kl_genenames
        lfc_data_spec.set_index('kl_genename',inplace = True)
    lfc_data_spec.index.name = spec.lower() + "_genename"
    
    lfc_data[spec] = lfc_data_spec

    #     fname_rlog_data = data_processing_dir + '20170817_klscpka/20170817_klscpka_avg_rlog_' + spec + '.csv'
    #     rlog_data[spec] = pd.read_csv(fname_rlog_data,index_col = 0)
    #     fname_raw_data = data_processing_dir + '20170817_klscpka/20170817_klscpka_avg_counts_' + spec + '.csv'
    #     raw_data[spec] = pd.read_csv(fname_raw_data,index_col = 0)

    #     #These are the genes that have no rlog value
    #     print('The following ' + str(len(set(lfc_data[spec].index)-set(rlog_data[spec].index))) + ' genes have no rlog data for ' + spec)
    #     print(set(lfc_data[spec].index)-set(rlog_data[spec].index))

    #     #Merge into one dataset
    #     labels_mean = [condition + "_mean" for condition in conditions]
    #     labels_var = [condition + "_var" for condition in conditions]
    #     labels = list(chain.from_iterable([labels_mean,labels_var]))

    #     rlog_subset = rlog_data[spec].loc[:,labels]
    #     rlog_subset.rename(index = str, columns = {label: 'rlog_' + label for label in labels},inplace=True)

    #     raw_subset = raw_data[spec].loc[:,labels]
    #     raw_subset.rename(index = str, columns = {label: 'raw_' + label for label in labels},inplace=True)
    
    comparison_data[spec] =  lfc_data_spec  #pd.concat([lfc_data[spec], rlog_subset , raw_subset],axis = 1)

#Map KL PKA inhibition data onto related SC orthologs
kl_comparison_data_sc_orthologs = kl_orthologs.merge(comparison_data['KL'], left_on='kl_genename', right_index=True, how = 'inner')

#add in SC_common_name to SC data
sc_comparison_data=comparison_data['SC']
sc_comparison_data['SC_common_name']= io_library.SC_common_name_lookup(sc_comparison_data.index)
#Just noticed that IMP2 has a common name IMP2' - should troubleshoot that someday. 

kl_sc_PKA_data = kl_comparison_data_sc_orthologs.merge(sc_comparison_data, on = 'sc_genename', suffixes=('_KL','_SC'), how='inner')
kl_sc_PKA_data
# fname_cross_spec_comparison = data_processing_dir + os.path.normpath("20170817_klscpka/cross_species_compare_ASminus_ASplus_YPD.csv")
# kl_sc_PKA_data.to_csv(fname_cross_spec_comparison)


,N SCer Orthologs,kl_genename,sc_genename,baseMean_KL,log2FoldChange_KL,lfcSE_KL,stat_KL,pvalue_KL,padj_KL,baseMean_SC,log2FoldChange_SC,lfcSE_SC,stat_SC,pvalue_SC,padj_SC,SC_common_name
0,1,KLLA0E08273g,YIL093C,42.467256,-3.450025,0.595751,-5.791055,6.994569e-09,6.303763e-08,121.871258,0.697651,0.450931,1.547136,1.218304e-01,2.019265e-01,RSM25
1,1,KLLA0E02179g,YNL323W,155.747290,-0.797739,0.365083,-2.185088,2.888238e-02,6.527648e-02,26.823746,-0.496615,0.497762,-0.997697,3.184265e-01,4.398740e-01,LEM3
2,1,KLLA0F12056g,YKR076W,172.696511,5.297480,0.469437,11.284757,1.560719e-29,1.033926e-27,177.964154,5.007464,0.391862,12.778653,2.157741e-37,7.127985e-36,ECM4
3,1,KLLA0E19691g,YDL174C,5.922864,-0.148512,0.918579,-0.161676,8.715611e-01,9.217983e-01,891.842439,2.506826,0.254273,9.858787,6.280348e-23,9.400896e-22,DLD1
4,1,KLLA0F20273g,YOR163W,68.694711,0.982368,0.412029,2.384222,1.711528e-02,4.175278e-02,281.856347,2.235830,0.299679,7.460747,8.603358e-14,7.208761e-13,DDP1
5,1,KLLA0D06127g,YOR246C,55.742604,-1.856236,0.491558,-3.776233,1.592180e-04,6.740007e-04,244.114300,-1.706814,0.301902,-5.653544,1.571729e-08,8.779689e-08,ENV9
6,1,KLLA0E07063g,YOL137W,30.037352,-2.693344,0.715948,-3.761928,1.686084e-04,7.108031e-04,130.752070,-0.854106,0.355581,-2.402003,1.630557e-02,3.554206e-02,BSC6
7,1,KLLA0F04499g,YNL067W,116.041800,-3.801339,0.481631,-7.892643,2.958532e-15,5.760103e-14,1061.744945,-3.318184,0.258310,-12.845726,9.089510e-38,3.073324e-36,RPL9B
8,1,KLLA0C15257g,YDR237W,132.305978,-3.661528,0.429372,-8.527629,1.493783e-17,3.735191e-16,173.485314,-0.174990,0.344939,-0.507306,6.119399e-01,7.171113e-01,MRPL7
9,1,KLLA0E08867g,YNL015W,51.954857,2.359815,0.562377,4.196142,2.715002e-05,1.345891e-04,87.194983,4.164874,0.445035,9.358534,8.085064e-21,1.051424e-19,PBI2


In [8]:
data = []

xlabel = 'LFC AS -/+ K.Lac'
ylabel = 'LFC AS -/+ S.Cer'

wgh_n_orthologs = {'Not WGH Paralog':1, 'WGH Paralog':2}
wgh_color = {'WGH Paralog':'red', 'Not WGH Paralog':'gray'}

for geneset in wgh_n_orthologs.keys():
    
    kl_sc_PKA_data_subset = kl_sc_PKA_data[kl_sc_PKA_data['N SCer Orthologs']==wgh_n_orthologs[geneset]]
    
    trace = pygo.Scatter(
                x = kl_sc_PKA_data_subset['log2FoldChange_KL'], 
                y =  kl_sc_PKA_data_subset['log2FoldChange_SC'],
                text = kl_sc_PKA_data_subset['SC_common_name'],
                mode = 'markers',
                marker = {'opacity': 0.5,  
                          'color': wgh_color[geneset],
                          'size': 10}, 
                name = geneset
            )

    data.append(trace)

layout = pygo.Layout(
    xaxis= {
        #"range":[-20, 20],
        "title": xlabel
    },
    yaxis= {
        #"range":[-10, 10],
        "title": ylabel
    }
)


fig = pygo.Figure(data=data, layout = layout)
py.iplot(fig, filename= spec + '_AS_pmNMPP1_kl_v_sc')

In [9]:
msn24_basal = ['YGL170C',
 'YMR182W-A',
 'YDR014W-A',
 'YAR029W',
 'YJL112W',
 'YMR081C',
 'YER053C',
 'YHR087W',
 'YDR343C',
 'YPL027W',
 'YCR025C',
 'YCR021C',
 'YJL181W',
 'YLR257W',
 'YFL059W',
 'YHL045W',
 'YOR049C',
 'YFR023W',
 'YER150W',
 'YIL059C',
 'YCR018C-A',
 'YKL187C',
 'YOR313C',
 'YOL052C-A',
 'YOR348C',
 'YHL021C',
 'YPL014W',
 'YEL039C',
 'YNR014W',
 'YPL171C',
 'YPL230W',
 'YGR008C',
 'YGL037C',
 'YGR035C',
 'YNL093W',
 'YDL022W',
 'YEL011W',
 'YBR047W',
 'YHR153C',
 'YLR312C',
 'YOR357C',
 'YNR034W-A',
 'YLR031W',
 'YDR171W',
 'YDR513W',
 'YBL029C-A',
 'YER067W']

msn24_pka_int = ['YCR045C',
 'YOR173W',
 'YHR097C',
 'YAL054C',
 'YMR133W',
 'YDR453C',
 'YJR096W',
 'YDR536W',
 'YER121W',
 'YMR118C',
 'YKL065W-A',
 'YML099W-A',
 'YML100W',
 'YFL030W',
 'YNL093W',
 'YBR116C',
 'YBL075C',
 'YKL151C',
 'YMR174C',
 'YDL223C',
 'YHR087W',
 'YMR107W',
 'YPL223C',
 'YBR117C',
 'YJR155W',
 'YMR090W',
 'YLR164W',
 'YCR083W',
 'YDR256C',
 'YHR139C',
 'YGR256W',
 'YDR193W',
 'YBR186W',
 'YFL015C',
 'YGR174W-A',
 'YDR119W-A',
 'YDR034W-B',
 'YNL195C',
 'YJL045W',
 'YPL186C',
 'YHR193C-A',
 'YBR072W',
 'YGR043C',
 'YMR169C',
 'YIL136W',
 'YER103W',
 'YMR206W',
 'YDL204W',
 'YDR070C',
 'YER084W-A',
 'YGR201C',
 'YGR248W',
 'YBR221W-A',
 'YLR311C',
 'YDR506C',
 'YOR120W',
 'YMR105C',
 'YDR491C',
 'YPL036W',
 'YDL222C',
 'YHR096C',
 'YOL084W',
 'YOR382W',
 'YMR175W',
 'YDR533C',
 'YDL085W',
 'YKL163W',
 'YKL187C',
 'YOL052C-A',
 'YCL048W-A',
 'YDR048C',
 'YDR010C',
 'YIL160C',
 'YER068C-A',
 'YGR008C',
 'YML128C',
 'YGR088W',
 'YNR034W-A']

In [14]:
kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(set(msn24_basal))]

,N SCer Orthologs,kl_genename,sc_genename,baseMean_KL,log2FoldChange_KL,lfcSE_KL,stat_KL,pvalue_KL,padj_KL,baseMean_SC,log2FoldChange_SC,lfcSE_SC,stat_SC,pvalue_SC,padj_SC,SC_common_name
100,1,KLLA0A10527g,YBR047W,12.155893,2.925677,0.833816,3.508780,4.501674e-04,1.731753e-03,8.182152,2.088461,1.025136,2.037253,4.162465e-02,8.163033e-02,FMP23
234,1,KLLA0F21736g,YHR087W,421.346144,1.301971,0.455228,2.860042,4.235853e-03,1.267278e-02,7734.889915,6.740608,0.409332,16.467332,6.298781e-61,7.542790e-59,RTC3
372,1,KLLA0D14553g,YHL021C,508.782773,4.834512,0.421386,11.472888,1.805312e-30,1.297174e-28,59.332813,2.891058,0.639323,4.522060,6.124080e-06,2.539770e-05,AIM17
687,1,KLLA0F16082g,YOR049C,0.262058,1.802512,4.058022,0.444185,6.569088e-01,7.697891e-01,35.940729,3.049146,0.501059,6.085408,1.161953e-09,7.275493e-09,RSB1
899,1,KLLA0F23419g,YOR348C,63.585710,-2.290319,0.593943,-3.856128,1.151972e-04,5.039632e-04,308.363789,5.250667,0.592812,8.857227,8.202964e-19,9.411304e-18,PUT4
1324,1,KLLA0F08679g,YOR313C,0.379528,0.082232,3.484273,0.023601,9.811711e-01,9.893168e-01,1.223903,-1.249250,1.962524,-0.636553,5.244163e-01,6.406685e-01,SPS4
1897,1,KLLA0F22242g,YGL037C,378.614243,4.544800,0.389357,11.672579,1.760057e-31,1.402821e-29,2378.780325,4.284569,0.241395,17.749180,1.748897e-70,3.866408e-68,PNC1
2199,1,KLLA0F27225g,YPL014W,104.788833,0.958324,0.462186,2.073460,3.812950e-02,8.287114e-02,143.343790,1.671195,0.388829,4.298023,1.723283e-05,6.679320e-05,YPL014W
2287,1,KLLA0D17446g,YDR171W,279.385355,3.471129,0.412146,8.422077,3.698690e-17,8.654595e-16,519.036787,3.522817,0.378799,9.299966,1.404904e-20,1.798527e-19,HSP42
2311,1,KLLA0A11176g,YEL011W,280.840406,4.410052,0.492647,8.951748,3.498970e-19,9.915692e-18,495.619904,7.118851,0.497572,14.307183,1.973452e-46,1.134340e-44,GLC3


In [32]:

len(set(msn24_basal) | set(msn24_pka_int))
#len(set(msn24_basal)-set(msn24_pka_int))
#len(set(msn24_pka_int)-set(msn24_basal))

119

In [33]:
data = []

spec = 'SC'

xlabel = 'LFC AS -/+ K.Lac'
ylabel = 'LFC AS -/+ S.Cer'

gene_subset = {'Neither': set(kl_sc_PKA_data['sc_genename']) - (set(msn24_basal) | set(msn24_pka_int)),
               'Msn2/4 Basal Activation': set(msn24_basal)-set(msn24_pka_int), 
               'Msn2/4 PKA activation': set(msn24_pka_int)-set(msn24_basal), 
               'Both': set(msn24_basal) & set(msn24_pka_int)
              }
               
colors = {'Msn2/4 Basal Activation':'green', 
             'Msn2/4 PKA activation':'red', 
            'Both': 'orange', 
            'Neither': 'gray'}

for geneset_name, geneset in gene_subset.items():
    
    kl_sc_PKA_data_subset = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(geneset)]
    
    trace = pygo.Scatter(
                x = kl_sc_PKA_data_subset['log2FoldChange_KL'], 
                y =  kl_sc_PKA_data_subset['log2FoldChange_SC'],
                text = kl_sc_PKA_data_subset['SC_common_name'],
                mode = 'markers',
                marker = {'opacity': 0.5,  
                          'color': colors[geneset_name],
                          'size': 10}, 
                name = geneset_name
            )

    data.append(trace)

layout = pygo.Layout(
    xaxis= {
        #"range":[-20, 20],
        "title": xlabel
    },
    yaxis= {
        #"range":[-10, 10],
        "title": ylabel
    }
)


fig = pygo.Figure(data=data, layout = layout)
py.iplot(fig, filename= 'AS_pmNMPP1_kl_v_sc_m24_targets_'+spec)

In [34]:
msn24_basal = {'KLLA0A00308r',
 'KLLA0A05665g',
 'KLLA0B09878g',
 'KLLA0B14212r',
 'KLLA0C00242g',
 'KLLA0C12034g',
 'KLLA0C16819g',
 'KLLA0C17886r',
 'KLLA0D03102r',
 'KLLA0D09845r',
 'KLLA0D14465r',
 'KLLA0D15477r',
 'KLLA0D19949g',
 'KLLA0E01079g',
 'KLLA0E03983g',
 'KLLA0E04951r',
 'KLLA0E05611g',
 'KLLA0E12541g',
 'KLLA0E14081r',
 'KLLA0E15005g',
 'KLLA0E15643r',
 'KLLA0E16809r',
 'KLLA0E20505g',
 'KLLA0F03212g',
 'KLLA0F13332g',
 'KLLA0F20097r',
 'KLLA0F24486g',
 'KLLA0F26961g'}

msn24_pka_int = {'EFG00000981736',
 'EFG00000982152',
 'EFG00000982249',
 'KLLA0A00484g',
 'KLLA0A02585g',
 'KLLA0A02629g',
 'KLLA0A02717g',
 'KLLA0A03289g',
 'KLLA0A04323g',
 'KLLA0A05852g',
 'KLLA0A05987r',
 'KLLA0A07227g',
 'KLLA0A08987r',
 'KLLA0A10087g',
 'KLLA0A11935g',
 'KLLA0B01430r',
 'KLLA0B04477g',
 'KLLA0B05269g',
 'KLLA0B10142g',
 'KLLA0B11946g',
 'KLLA0B12650g',
 'KLLA0B14751g',
 'KLLA0C01177g',
 'KLLA0C02079g',
 'KLLA0C02409g',
 'KLLA0C03938g',
 'KLLA0C04686g',
 'KLLA0C05940g',
 'KLLA0C06490g',
 'KLLA0C06996g',
 'KLLA0C08866g',
 'KLLA0C11781g',
 'KLLA0C16599g',
 'KLLA0C17237g',
 'KLLA0C18722g',
 'KLLA0D02376g',
 'KLLA0D03124r',
 'KLLA0D03410g',
 'KLLA0D03751g',
 'KLLA0D04037g',
 'KLLA0D05907g',
 'KLLA0D07073g',
 'KLLA0D07656g',
 'KLLA0D13046g',
 'KLLA0D16412g',
 'KLLA0D17248g',
 'KLLA0D18172g',
 'KLLA0D18282g',
 'KLLA0D18964g',
 'KLLA0E00419g',
 'KLLA0E00661g',
 'KLLA0E01893g',
 'KLLA0E05875g',
 'KLLA0E06051g',
 'KLLA0E06183g',
 'KLLA0E06711g',
 'KLLA0E07327g',
 'KLLA0E07525g',
 'KLLA0E07613g',
 'KLLA0E12035g',
 'KLLA0E12827g',
 'KLLA0E14235g',
 'KLLA0E15027g',
 'KLLA0E15753g',
 'KLLA0E15929r',
 'KLLA0E18415g',
 'KLLA0E18635g',
 'KLLA0E19295g',
 'KLLA0E19317g',
 'KLLA0E19493g',
 'KLLA0E20527g',
 'KLLA0E20725g',
 'KLLA0E22177g',
 'KLLA0E25180g',
 'KLLA0F00374g',
 'KLLA0F03773g',
 'KLLA0F04433g',
 'KLLA0F05203g',
 'KLLA0F05533r',
 'KLLA0F06512g',
 'KLLA0F07348g',
 'KLLA0F07601g',
 'KLLA0F07843g',
 'KLLA0F08745g',
 'KLLA0F09295g',
 'KLLA0F14344g',
 'KLLA0F14531g',
 'KLLA0F14553g',
 'KLLA0F16104g',
 'KLLA0F17050g',
 'KLLA0F19921g',
 'KLLA0F22341g',
 'KLLA0F25520g',
 'KLLA0F25916g',
 'KLLA0F27203g',
 'KLLA0F27819g'}

In [36]:
kl_sc_PKA_data[kl_sc_PKA_data['SC_common_name']=='GRE1']

,N SCer Orthologs,kl_genename,sc_genename,baseMean_KL,log2FoldChange_KL,lfcSE_KL,stat_KL,pvalue_KL,padj_KL,baseMean_SC,log2FoldChange_SC,lfcSE_SC,stat_SC,pvalue_SC,padj_SC,SC_common_name
4304,2,KLLA0C16225g,YPL223C,50.58652,9.239796,1.270372,7.273301,3.508070e-13,5.422626e-12,2994.775471,9.563095,1.211634,7.892725,2.956582e-15,2.732224e-14,GRE1


In [35]:
data = []

spec = 'KL'

xlabel = 'LFC AS -/+ K.Lac'
ylabel = 'LFC AS -/+ S.Cer'

gene_subset = {'Neither': set(kl_sc_PKA_data['kl_genename']) - (set(msn24_basal) | set(msn24_pka_int)),
               'Msn2/4 Basal Activation': set(msn24_basal)-set(msn24_pka_int), 
               'Msn2/4 PKA activation': set(msn24_pka_int)-set(msn24_basal), 
               'Both': set(msn24_basal) & set(msn24_pka_int)
              }
               
colors = {'Msn2/4 Basal Activation':'green', 
             'Msn2/4 PKA activation':'red', 
            'Both': 'orange', 
            'Neither': 'gray'}

for geneset_name, geneset in gene_subset.items():
    
    kl_sc_PKA_data_subset = kl_sc_PKA_data[kl_sc_PKA_data['kl_genename'].isin(geneset)]
    
    trace = pygo.Scatter(
                x = kl_sc_PKA_data_subset['log2FoldChange_KL'], 
                y =  kl_sc_PKA_data_subset['log2FoldChange_SC'],
                text = kl_sc_PKA_data_subset['SC_common_name'],
                mode = 'markers',
                marker = {'opacity': 0.5,  
                          'color': colors[geneset_name],
                          'size': 10}, 
                name = geneset_name
            )

    data.append(trace)

layout = pygo.Layout(
    xaxis= {
        #"range":[-20, 20],
        "title": xlabel
    },
    yaxis= {
        #"range":[-10, 10],
        "title": ylabel
    }
)


fig = pygo.Figure(data=data, layout = layout)
py.iplot(fig, filename= 'AS_pmNMPP1_kl_v_sc_m24_targets_'+spec)

In [43]:
sc_rg_rep = {'YAL005C',
 'YBR006W',
 'YBR054W',
 'YBR072W',
 'YBR111W-A',
 'YBR183W',
 'YBR214W',
 'YBR269C',
 'YBR280C',
 'YBR285W',
 'YCL012C',
 'YCL033C',
 'YCL044C',
 'YCR025C',
 'YCR083W',
 'YDL021W',
 'YDL022W',
 'YDL085W',
 'YDL110C',
 'YDL130W-A',
 'YDL169C',
 'YDR178W',
 'YDR277C',
 'YDR358W',
 'YDR379C-A',
 'YDR461W',
 'YDR470C',
 'YDR513W',
 'YDR516C',
 'YEL012W',
 'YEL039C',
 'YER011W',
 'YER035W',
 'YER037W',
 'YER039C',
 'YER053C-A',
 'YER062C',
 'YER096W',
 'YGL032C',
 'YGL037C',
 'YGL156W',
 'YGL180W',
 'YGL227W',
 'YGR052W',
 'YGR053C',
 'YGR066C',
 'YGR088W',
 'YGR149W',
 'YGR194C',
 'YGR222W',
 'YHL044W',
 'YHL045W',
 'YHR008C',
 'YHR092C',
 'YHR096C',
 'YHR113W',
 'YHR140W',
 'YHR171W',
 'YHR199C-A',
 'YIL055C',
 'YIL099W',
 'YIL100C-A',
 'YIL101C',
 'YIL113W',
 'YIR017C',
 'YIR037W',
 'YJL062W-A',
 'YJL070C',
 'YJL089W',
 'YJL163C',
 'YJL164C',
 'YJR019C',
 'YKL062W',
 'YKL103C',
 'YKL123W',
 'YKL148C',
 'YKR011C',
 'YKR098C',
 'YLL041C',
 'YLR012C',
 'YLR030W',
 'YLR031W',
 'YLR108C',
 'YLR149C',
 'YLR178C',
 'YLR290C',
 'YLR297W',
 'YLR324W',
 'YLR345W',
 'YLR356W',
 'YLR363C',
 'YML089C',
 'YMR025W',
 'YMR053C',
 'YMR105C',
 'YMR175W',
 'YMR181C',
 'YMR250W',
 'YMR319C',
 'YNL011C',
 'YNL014W',
 'YNL015W',
 'YNL092W',
 'YNL115C',
 'YNL160W',
 'YNL194C',
 'YNL237W',
 'YNL279W',
 'YOL052C-A',
 'YOL083W',
 'YOL151W',
 'YOR020W-A',
 'YOR161C',
 'YOR185C',
 'YPL003W',
 'YPL054W',
 'YPL123C',
 'YPL154C',
 'YPL203W',
 'YPL223C',
 'YPR026W',
 'YPR098C',
 'YPR192W'}


In [42]:

kl_rg_rep_kl = ['EFG00000981672',
 'EFG00000982249',
 'KLLA0A00220g',
 'KLLA0A01320g',
 'KLLA0A11638g',
 'KLLA0A11836g',
 'KLLA0B00451g',
 'KLLA0B01243g',
 'KLLA0B02464g',
 'KLLA0B05533g',
 'KLLA0B07337g',
 'KLLA0B07711g',
 'KLLA0B08349r',
 'KLLA0B12650g',
 'KLLA0C00671g',
 'KLLA0C06886g',
 'KLLA0C08085r',
 'KLLA0C08173g',
 'KLLA0C11121g',
 'KLLA0C11209g',
 'KLLA0C12463g',
 'KLLA0C13882g',
 'KLLA0C14916g',
 'KLLA0C19019g',
 'KLLA0C19041g',
 'KLLA0D00462g',
 'KLLA0D00704g',
 'KLLA0D02464g',
 'KLLA0D05907g',
 'KLLA0D06083g',
 'KLLA0D13046g',
 'KLLA0D17248g',
 'KLLA0D18909g',
 'KLLA0E01563g',
 'KLLA0E07877g',
 'KLLA0E08867g',
 'KLLA0E10539g',
 'KLLA0E10935g',
 'KLLA0E14477g',
 'KLLA0E15203g',
 'KLLA0E15401g',
 'KLLA0E15709g',
 'KLLA0E19493g',
 'KLLA0E21319g',
 'KLLA0E22551g',
 'KLLA0F00682g',
 'KLLA0F01507g',
 'KLLA0F03773g',
 'KLLA0F04059g',
 'KLLA0F04125g',
 'KLLA0F04323g',
 'KLLA0F05335g',
 'KLLA0F08107g',
 'KLLA0F10065g',
 'KLLA0F12056g',
 'KLLA0F20768g',
 'KLLA0F20878g',
 'KLLA0F20900g',
 'KLLA0F22275g',
 'KLLA0F23914g',
 'KLLA0F25520g',
 'KLLA0F26829g',
 'KLLA0F27203g']

kl_rg_rep_sc = set(kl_sc_PKA_data[kl_sc_PKA_data['kl_genename'].isin(kl_rg_rep_kl)].loc[:,'sc_genename'])

In [45]:
io_library.SC_common_name_lookup(list(sc_rg_rep & kl_rg_rep_sc))

['UGX2', 'PBI2', 'YLR149C', 'YNL194C', 'YIL055C', 'SGA1']

In [48]:
data = []

spec = 'KL_SC'

xlabel = 'LFC AS -/+ K.Lac'
ylabel = 'LFC AS -/+ S.Cer'

gene_subset = {'Neither': set(kl_sc_PKA_data['sc_genename']) - (sc_rg_rep | kl_rg_rep_sc),
               'SCer Only': sc_rg_rep-kl_rg_rep_sc, 
               'KLac Only': kl_rg_rep_sc-sc_rg_rep, 
               'Both': sc_rg_rep & kl_rg_rep_sc
              }
               
colors = {'SCer Only':'red', 
             'KLac Only':'blue', 
            'Both': 'purple', 
            'Neither': 'gray'}

for geneset_name, geneset in gene_subset.items():
    
    kl_sc_PKA_data_subset = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(geneset)]
    
    trace = pygo.Scatter(
                x = kl_sc_PKA_data_subset['log2FoldChange_KL'], 
                y =  kl_sc_PKA_data_subset['log2FoldChange_SC'],
                text = kl_sc_PKA_data_subset['SC_common_name'],
                mode = 'markers',
                marker = {'opacity': 0.5,  
                          'color': colors[geneset_name],
                          'size': 10}, 
                name = geneset_name
            )

    data.append(trace)

layout = pygo.Layout(
    xaxis= {
        #"range":[-20, 20],
        "title": xlabel
    },
    yaxis= {
        #"range":[-10, 10],
        "title": ylabel
    }
)


fig = pygo.Figure(data=data, layout = layout)
py.iplot(fig, filename= 'AS_pmNMPP1_kl_v_sc_rg_targets_'+spec)